In [1]:
import rospy
from sensor_msgs.msg import CompressedImage, Image
from geometry_msgs.msg import PoseStamped, TransformStamped
from nav_msgs.msg import Odometry
from cv_bridge.boost.cv_bridge_boost import getCvType
from cv_bridge import CvBridge, CvBridgeError
import tf2_ros
import tf

from traj_utils.msg import CirclePoses

In [2]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import time

from draw import plot3d
from reproj import SFM, FRAME, p2w_w2p_once, calc_circle

In [3]:
def load_frame(idx):
    data = np.load('/home/c/path-v1/sfm2_data/left_{}.npz'.format(idx))
    img = data['img']
    pos = data['pos']
    R = data['R']
    data.close()
    
    return img, R, pos, None

In [4]:
from reproj import w2p_sim, KRt2proj, init_circle

In [5]:
origin, vec, yaw = np.float32([[23.211326599121094, 6.285023212432861, -0.5427668690681458]]), [0, 0, 1], -0.07842210461956332
sfm = SFM(origin, vec, yaw)
img, R, pos, target = load_frame(950)
frame = FRAME(img, R, pos, thresh = 40.0)
wt = frame.p2w_v2(sfm, frame.cnt)
wt = p2w_w2p_once(sfm, frame, wt)
origin_, vec_, yaw_, theta, radius = calc_circle(wt)
sfm.update_sfm(origin_, vec_, yaw_)
print(origin_, vec_, yaw_, radius)
if 1:
    KRt = KRt2proj(sfm.CK, np.linalg.inv(frame.R), np.linalg.inv(frame.R).dot(-(frame.pos + sfm.CAM_T)))
    pt2d = w2p_sim(KRt, wt)
    img3 = cv2.drawContours(frame.img.copy(), [np.int32(pt2d)], -1, (0,255,0), 3)
    outer, _ = init_circle(origin_, radius, vec_, yaw_, theta)
    _, _, outer_img = frame.w2p(sfm, outer)
    img1 = cv2.drawContours(frame.img.copy(), [np.int32(outer_img)], -1, (0,255,255), 3)
    img2 = cv2.drawContours(frame.img.copy(), [np.int32(frame.cnt)], -1, (255,0,0), 3)
    plt.figure(figsize = (10, 5))
    plt.subplot(131)
    plt.imshow(img1)
    plt.subplot(132)
    plt.imshow(img2)
    plt.subplot(133)
    plt.imshow(img3)
    plt.show()
plot3d(outer)
plot3d(wt)

TypeError: __init__() missing 1 required positional argument: 'pos'

In [ ]:
origin, vec, yaw = np.float32([[23.211326599121094, 6.285023212432861, -0.5427668690681458]]), [0, 0, 1], -0.07842210461956332
sfm = SFM(origin, vec, yaw)
#choose = np.random.randint(800, 960, 2)

length = 100
theta = np.arange(length) * 2 * np.pi / length - np.pi
outer, _ = init_circle(origin, 0.78, vec, yaw, theta)

for idx in range(800, 950):
    img, R, pos, target = load_frame(idx)
    print(R)
    print(np.linalg.det(R), pos)
    frame = FRAME(img, R, pos, thresh = 20.0)
    _, _, pix = frame.w2p(sfm, outer)
    img_ = cv2.drawContours(frame.img.copy(), [np.int32(pix)], -1, (255,0,0), 3)
    plt.imshow(img_)
    plt.show()
    break

In [ ]:
origin, vec, yaw = np.float32([[23.211326599121094, 6.285023212432861, -0.5427668690681458]]), [0, 0, 1], -0.07842210461956332
sfm = SFM(origin, vec, yaw)
for idx in range(850, 950):
    img, R, pos, target = load_frame(idx)
    frame = FRAME(img, R, pos, thresh = 40.0)
    if frame.ch == -1: continue
    print(idx, "==============")
    wt = frame.p2w_v2(sfm, frame.cnt)
    wt = p2w_w2p_once(sfm, frame, wt)
    origin_, vec_, yaw_, theta, radius = calc_circle(wt)
    sfm.update_sfm(origin_, vec_, yaw_)
    print(origin_, vec_, yaw_, radius)
    if 1:
        KRt = KRt2proj(sfm.CK, np.linalg.inv(frame.R), np.linalg.inv(frame.R).dot(-(frame.pos + sfm.CAM_T)))
        pt2d = w2p_sim(KRt, wt)
        img3 = cv2.drawContours(frame.img.copy(), [np.int32(pt2d)], -1, (0,255,0), 3)
        outer, _ = init_circle(origin_, radius, vec_, yaw_, theta)
        _, _, outer_img = frame.w2p(sfm, outer)
        img1 = cv2.drawContours(frame.img.copy(), [np.int32(outer_img)], -1, (0,255,255), 3)
        img2 = cv2.drawContours(frame.img.copy(), [np.int32(frame.cnt)], -1, (255,0,0), 3)
        plt.figure(figsize = (10, 5))
        plt.subplot(131)
        plt.imshow(img1)
        plt.subplot(132)
        plt.imshow(img2)
        plt.subplot(133)
        plt.imshow(img3)
        plt.show()
    plot3d(wt)
   